# Hamming Code

This notebook contains my playing around with Hamming Code Error correction
The staer is inspired by 3Blue1Brown, but i seek to build on that for scaling and 

In [1]:
import operator as op
import numpy as np
import random
from functools import reduce
import math
import itertools

In [2]:
bits = np.random.randint(0,2,16)

In [3]:
bits

array([1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1])

In [4]:
list(enumerate(bits))

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 0),
 (6, 0),
 (7, 1),
 (8, 0),
 (9, 1),
 (10, 1),
 (11, 0),
 (12, 1),
 (13, 0),
 (14, 0),
 (15, 1)]

In [5]:
[i for i , bit in enumerate(bits)]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

In [6]:
[i for i , bit in enumerate(bits) if bit]

[0, 1, 2, 3, 4, 7, 9, 10, 12, 15]

In [7]:
[bit for i , bit in enumerate(bits) if bit]

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [8]:
reduce(op.xor, [i for i , bit in enumerate(bits) if bit])

3

In [9]:
bin(reduce(op.xor, [i for i , bit in enumerate(bits) if bit]))

'0b11'

Manipulate bitstring to get the parity = 0

In [10]:
bits[8] = not bits[8]
bits[1] = not bits[1]

In [11]:
reduce(op.xor, [i for i , bit in enumerate(bits) if bit])

10

add an error

In [12]:
bits[10] = not bits[10]

In [13]:
reduce(op.xor, [i for i , bit in enumerate(bits) if bit])

0

Function from 3Blue1Brown

In [2]:
def Hamming_syndrome(bits):
    return reduce(op.xor, [i for i , b in enumerate(bits) if b])

In [15]:
Hamming_syndrome(bits)

0

## Start om my own work
Up until now I followed along with the 3Blue1Brown series on the Hamming code. Going on from here, I plan to look at how to scale and implement it on an arbitrary sised array.

### Create the Hamming encoded bitstring

The initial step is to create a function that takes an arbitrary* bites length of data end encodes it with the Hamming Parity bites.

*= constrained to powers of 2

In [3]:
#check if value is a power of 2
def power_of_two(n): 
    if (n == 0): 
        return False
    while (n != 1): 
            if (n % 2 != 0): 
                return False
            n = n // 2
              
    return True

In [17]:
power_of_two(16)

True

In [18]:
# print the vlaue for bit possitions with the value power 2
for position, bit  in enumerate(bits):
    if power_of_two(position)==True:
        print (bit)

0
0
1
0


For a 16 bit hamming mesage we have 11 information carying bits, the rest are redundancy

In [19]:
#create a message to encode
message = np.random.randint(0,2,11)
message

array([1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0])

In [20]:
block = 16
for position in range(1, block+1):
    print(power_of_two(position))

True
True
False
True
False
False
False
True
False
False
False
False
False
False
False
True


In [21]:
power_of_two(0)

False

In [22]:
encoded_message = []
message_position = 0
for position in range(0, block):
    if position==0:
        encoded_message.append(0)
    elif power_of_two(position)==True:
        encoded_message.append(0)
    else:
        encoded_message.append(message[message_position])
        message_position =message_position+ 1

In [23]:
print(encoded_message)
print(message)

[0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0]
[1 0 1 1 1 0 0 1 0 1 0]


In [24]:
list(enumerate(encoded_message))

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 0),
 (6, 1),
 (7, 1),
 (8, 0),
 (9, 1),
 (10, 0),
 (11, 0),
 (12, 1),
 (13, 0),
 (14, 1),
 (15, 0)]

In [25]:
[i for i , bit in enumerate(bits) if (i%2)!=0]

[1, 3, 5, 7, 9, 11, 13, 15]

In [26]:
parity_group_1 = [bit for i , bit in enumerate(encoded_message) if (i%2)!=0]
parity_group_1

[0, 1, 0, 1, 1, 0, 0, 0]

In [27]:
if (parity_group_1[1:].count(1)%2==0)== False:
    parity_group_1[0] = 1
else:
    parity_group_1[0] = 0 

In [28]:
parity_group_2 = [bit for i, bit in enumerate(encoded_message) if (i%2)==0]

While this aproach works, it is not svclable, and we should utilise the fact that parity groups are based on the byte representation of the possition as done here

In [29]:
[bin(i)[2:] for i, bit in enumerate(encoded_message)]

['0',
 '1',
 '10',
 '11',
 '100',
 '101',
 '110',
 '111',
 '1000',
 '1001',
 '1010',
 '1011',
 '1100',
 '1101',
 '1110',
 '1111']

In [30]:
[bin(i)[2:].zfill(4) for i, bit in enumerate(encoded_message)]

['0000',
 '0001',
 '0010',
 '0011',
 '0100',
 '0101',
 '0110',
 '0111',
 '1000',
 '1001',
 '1010',
 '1011',
 '1100',
 '1101',
 '1110',
 '1111']

In [31]:
[bin(i)[2:].zfill(4)[3] for i, bit in enumerate(encoded_message)]

['0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1',
 '0',
 '1']

In [32]:
[bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[3] == '1'][1:]

[1, 0, 1, 1, 0, 0, 0]

In [33]:
[bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[3] == '1'][1:].count(1)

3

In [34]:
([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[3] == '1'][1:].count(1))%2==0

False

In [35]:
([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4) == '0100'])

[0]

In [36]:
list(enumerate(encoded_message))

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 0),
 (6, 1),
 (7, 1),
 (8, 0),
 (9, 1),
 (10, 0),
 (11, 0),
 (12, 1),
 (13, 0),
 (14, 1),
 (15, 0)]

In [37]:
for i, b in enumerate(encoded_message):
    if bin(i)[2:].zfill(4) == '0100':
        if (([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[1] == '1'][1:].count(1))%2==0) == True:
            encoded_message[i]= 0
        else:
            encoded_message[i]= 1

In [38]:
list(enumerate(encoded_message))

[(0, 0),
 (1, 0),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 0),
 (6, 1),
 (7, 1),
 (8, 0),
 (9, 1),
 (10, 0),
 (11, 0),
 (12, 1),
 (13, 0),
 (14, 1),
 (15, 0)]

In [39]:
for i, b in enumerate(encoded_message):
    if bin(i)[2:].zfill(4) == '1000':
        if (([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[0] == '1'][1:].count(1))%2==0) == True:
            encoded_message[i]= 0
        else:
            encoded_message[i]= 1
            
    elif bin(i)[2:].zfill(4) == '0100':
        if (([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[1] == '1'][1:].count(1))%2==0) == True:
            encoded_message[i]= 0
        else:
            encoded_message[i]= 1
            
    elif bin(i)[2:].zfill(4) == '0010':
        if (([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[2] == '1'][1:].count(1))%2==0) == True:
            encoded_message[i]= 0
        else:
            encoded_message[i]= 1
            
    elif bin(i)[2:].zfill(4) == '0001':
        if (([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[3] == '1'][1:].count(1))%2==0) == True:
            encoded_message[i]= 0
        else:
            encoded_message[i]= 1
    elif bin(i)[2:].zfill(4) == '0000':
        if (([bit for i, bit in enumerate(encoded_message)][1:].count(1))%2==0) == True:
            encoded_message[i]= 0
        else:
            encoded_message[i]= 1

In [40]:
list(enumerate(encoded_message))

[(0, 0),
 (1, 1),
 (2, 0),
 (3, 1),
 (4, 0),
 (5, 0),
 (6, 1),
 (7, 1),
 (8, 1),
 (9, 1),
 (10, 0),
 (11, 0),
 (12, 1),
 (13, 0),
 (14, 1),
 (15, 0)]

In [41]:
# Checking thhat the implementation creates a valid Hamming value
Hamming_syndrome(encoded_message)

0

In [7]:
# Create a new message, encode it and test
message = np.random.randint(0,2,11)
block=16

encoded_message = []
message_position = 0
for position in range(0, block):
    if position==0:
        encoded_message.append(0)
    elif power_of_two(position)==True:
        encoded_message.append(0)
    else:
        encoded_message.append(message[message_position])
        message_position =message_position+ 1

In [8]:
Hamming_syndrome(encoded_message)

6

In [9]:
for index in list(range(0, len(bin(list(enumerate(encoded_message))[-1][0])[2:]))):
    parity_byte = '0'* len(bin(list(enumerate(encoded_message))[-1][0])[2:])
    parity_byte = parity_byte[:index]+parity_byte[index].replace('0','1')+parity_byte[index:][:-1]
    for i, b in enumerate(encoded_message):
        if bin(i)[2:].zfill(4) == parity_byte:
            if (([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[index] == '1'][1:].count(1))%2==0) == True:
                encoded_message[i]= 0
            else:
                encoded_message[i]= 1

In [10]:
Hamming_syndrome(encoded_message)

0

### Function for Parity correction

Okay, Having worked thrugh how i got here it is time to put it into a function that alows us to take any sizes encoded message and turn it into a Hamming prepared code

In [9]:
def Hamming_code (encoded_message):
    for index in list(range(0, len(bin(list(enumerate(encoded_message))[-1][0])[2:]))):
        parity_byte = '0'* len(bin(list(enumerate(encoded_message))[-1][0])[2:])
        parity_byte = parity_byte[:index]+parity_byte[index].replace('0','1')+parity_byte[index:][:-1]
        for i, b in enumerate(encoded_message):
            if bin(i)[2:].zfill(4) == parity_byte:
                if (([bit for i, bit in enumerate(encoded_message) if bin(i)[2:].zfill(4)[index] == '1'][1:].count(1))%2==0) == True:
                    encoded_message[i]= 0
                else:
                    encoded_message[i]= 1
            elif bin(i)[2:].zfill(4) == ('0'* len(bin(list(enumerate(encoded_message))[-1][0])[2:])):
                if (([bit for i, bit in enumerate(encoded_message)][1:].count(1))%2==0) == True:
                    encoded_message[i]= 0
                else:
                    encoded_message[i]= 1
    
    
    return encoded_message

In [11]:
encoded_message = Hamming_code(encoded_message)

In [12]:
Hamming_syndrome(encoded_message)

0

### Function for encoding Message

This functions is created to take any length function, and encoding it into a hamming space

In [13]:
# Create a new message, encode it and test
message = np.random.randint(0,2,11)

encoded_message = []
message_position = 0
for position in range(0, block):
    if position==0:
        encoded_message.append(0)
    elif power_of_two(position)==True:
        encoded_message.append(0)
    else:
        encoded_message.append(message[message_position])
        message_position =message_position+ 1

In [5]:
def closest_power2(x):
    return 1<<(x-1).bit_length()

In [15]:
message = np.random.randint(0,2,34)
block = closest_power2(len(message))
block

64

In [16]:
pad = block - len(message)
pad

30

In [17]:
padding = [0] * pad
padded_message = np.append(message,padding)


In [18]:
len(padded_message)

64

In [19]:
print(padded_message)

[0 0 0 1 1 1 1 1 1 0 1 0 0 0 0 1 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [6]:
def Hamming_precode(message):
    block = closest_power2(len(message))
    pad = block - len(message)
    padding = [0] * pad
    padded_message = np.append(message,padding)
    
    encoded_message = []
    message_position = 0
    for position in range(0, block):
        if position==0:
            encoded_message.append(0)
        elif power_of_two(position)==True:
            encoded_message.append(0)
        else:
            encoded_message.append(padded_message[message_position])
            message_position += 1
    return encoded_message

In [7]:
message = np.random.randint(0,2,33)
precoded_message = Hamming_precode(message)
Hamming_syndrome(precoded_message)

14

In [10]:
encoded_message = Hamming_code(precoded_message)
Hamming_syndrome(encoded_message)

14